# Remove the holdout data

# Set up

In [1]:
import os
import sys

import pandas as pd
from dotenv import load_dotenv
from loguru import logger
from pydantic import BaseModel
from sqlalchemy import create_engine

sys.path.insert(0, "..")

load_dotenv("../.env")

True

# Controller

In [2]:
class Args(BaseModel):
    notebook_persist_dp: str = None
    random_seed: int = 41

    holdout_fp: str = "../../holdout.parquet"

    # Output PostgreSQL table
    table_name: str = "amz_review_rating_raw"


args = Args()

print(args.model_dump_json(indent=2))

{
  "notebook_persist_dp": null,
  "random_seed": 41,
  "holdout_fp": "../../holdout.parquet",
  "table_name": "amz_review_rating_raw"
}


# Load data

In [3]:
holdout_df = pd.read_parquet(args.holdout_fp).assign(
    description=lambda df: df["description"].apply(list),
    categories=lambda df: df["categories"].apply(list),
)
holdout_df

,user_id,parent_asin,rating,timestamp,main_category,title,description,categories,price
0,AHLBT2RDWYQWN5O2XNBNX2JPWVZA,B08NYV2VLS,4.0,2022-07-08 18:26:28.360,Video Games,Story of Seasons: Trio of Towns - Nintendo 3DS,[STORY OF SEASONS: Trio of Towns is a fresh ne...,"[Video Games, Legacy Systems, Nintendo Systems...",None
1,AHLBT2RDWYQWN5O2XNBNX2JPWVZA,B00KWIYPZG,5.0,2022-07-08 18:27:49.294,Video Games,Fantasy Life - 3DS,[Embark on a Journey that Lets You Build Your ...,"[Video Games, Legacy Systems, Nintendo Systems...",96.65
2,AF5NKVKUZGRPBR7HAYYDUS25RGRQ,B0BKRXQ5GL,3.0,2022-07-06 12:14:32.366,Computers,Logitech G Logitech G935 Over Ear Wireless Hea...,[Logitech G935 Wireless DTS:X 7.1 Surround Sou...,"[Video Games, PC, Accessories, Headsets]",153.98
3,AGGRGJRYYYWAL7V5M4RG4VFKL3HA,B07BGYLS1L,5.0,2022-06-25 20:06:42.077,Video Games,Shadow of the Tomb Raider - Xbox One,[Experience Lara croft's defining moment as sh...,"[Video Games, Xbox One, Games]",14.8
4,AEKEN3WITS4ZEJ7ZIISGJDZYJB3Q,B0BH1ZL3G9,5.0,2022-06-16 19:54:29.703,Computers,Hipshotdot PRO Color and Brightness Control Do...,[The HipShotDot is the gaming industry’s first...,"[Video Games, PC, Accessories, Controllers, Ga...",22.99
...,...,...,...,...,...,...,...,...,...
69,AETZPD7JKD42GBVYXBYPGOY4NF6Q,B09B35J159,4.0,2022-07-08 19:10:11.311,Computers,Razer Basilisk Ultimate HyperSpeed Wireless Ga...,"[With a high-speed transmission, extremely low...","[Video Games, PC, Accessories, Gaming Mice]",None
70,AETZPD7JKD42GBVYXBYPGOY4NF6Q,B09QVJDVHN,5.0,2022-07-08 19:13:28.232,Computers,Redragon K552 Mechanical Gaming Keyboard RGB L...,[Redragon K552 Pro KUMARA 87 Key RGB LED Backl...,"[Video Games, PC, Accessories, Gaming Keyboards]",48.99
71,AEVNAGOLV5MAGEPCCTT6ADCGNFWA,B08DF248LD,3.0,2022-07-08 05:07:25.394,Video Games,Xbox Core Wireless Controller – Carbon Black,[Experience the modernized design of the Xbox ...,[],45.5
72,AGMNHX4YCSY3BJTLDALLPD5XEYTA,B07WSHTJ48,3.0,2022-07-01 00:34:15.608,Computers,"Redragon S101 Gaming Keyboard, M601 Mouse, RGB...",[],"[Video Games, PC, Accessories, Gaming Keyboards]",39.99


# Remove holdout data from OLTP

In [4]:
# PostgreSQL connection details
username = os.getenv("POSTGRES_USER")
password = os.getenv("POSTGRES_PASSWORD")
host = os.getenv("POSTGRES_HOST")
port = os.getenv("POSTGRES_PORT")
database = os.getenv("POSTGRES_DB")
schema = os.getenv("POSTGRES_OLTP_SCHEMA")

# Create a connection string and engine outside the function
connection_string = f"postgresql://{username}:{password}@{host}:{port}/{database}"
engine = create_engine(connection_string)

In [5]:
def get_curr_oltp_max_timestamp():
    query = f"select max(timestamp) as max_timestamp from {schema}.{args.table_name};"
    return pd.read_sql(query, engine)["max_timestamp"].iloc[0]

In [6]:
assert get_curr_oltp_max_timestamp() == holdout_df["timestamp"].max()

In [7]:
from sqlalchemy import text

delete_query = f"DELETE FROM {schema}.{args.table_name} WHERE timestamp >= :timestamp;"
min_timestamp = holdout_df['timestamp'].min().strftime('%Y-%m-%d')

with engine.connect() as connection:
    with connection.begin():
        result = connection.execute(text(delete_query), {'timestamp': min_timestamp})
        logger.info(f"Deleted {result.rowcount} rows")

2024-10-14 00:43:14.939 | INFO     | __main__:<module>:9 - Deleted 74 rows


In [8]:
assert get_curr_oltp_max_timestamp() < holdout_df["timestamp"].min()